In [95]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np
import time
from tqdm.notebook import tqdm 

In [96]:
def parse_oneline(txt):
    output = [i for i in txt.split('\n')]
    return output

def parse_onepage(page_num):
    html=requests.get(f"https://www.khug.or.kr/jeonse/web/s07/s070102.jsp?cur_page={page_num}")
    tb_TF = False
    try_cnt=0
    while not tb_TF:
        if try_cnt>300:
            raise
        time.sleep(1) 
        bs_test = BeautifulSoup(html.text, 'html.parser')
        if bs_test.find('table'):
            tb_TF = True 
        else:
            tb_TF = False
        try_cnt+=1
        
    tables = bs_test.find('table')
    thead = tables.find('thead')
    tab_cols = parse_oneline(thead.get_text().strip())
    tbody = tables.find('tbody')
    output_tab = pd.DataFrame([parse_oneline(i.strip()) for i in tbody.get_text().strip().split('\n\n')], columns=tab_cols)
    output_tab = output_tab.assign(href_id = np.unique([re.search('no=\d{10}', i['href']).group() for i in tbody.find_all(href=True)]))
    return output_tab

In [97]:
list_num = 11
datas = [parse_onepage(n) for n in tqdm(range(1,list_num+1))]

  0%|          | 0/6 [00:00<?, ?it/s]

In [98]:
final_data = pd.concat(datas).reset_index(drop=True).assign(address = lambda df: df['주소'].apply(lambda x: x.split('  ')[0] if ',' not in x else  x.split('  ')[0].split(',')[0])
                                               # .apply(lambda x: ' '.join([i for i in x.split(' ') if i !='']))
                                              )

In [99]:
# def find_coordinates(address):
#     time.sleep(1)
#     apiurl = "https://api.vworld.kr/req/address?"
#     params = {
#     	"service": "address",
#     	"request": "getcoord",
#     	"crs": "epsg:5186",
#     	"address": address,
#     	"format": "json",
#     	"type": "parcel",
#     	"key": "{vworld-key}"
#     }
#     response = requests.get(apiurl, params=params)
#     if response.status_code == 200:
#         result = response.json()['response']
#         # print(result)
#         if 'result' in result.keys():
#             return result['result']['point']['x'], result['result']['point']['y']
#         else:
#             return 0,0


import requests
def set_rest_api(rest_api_token):
    def convert_address_to_coordinates(address):
        """
        입력받은 주소를 WGS84 좌표계 좌표로 변환(카카오맵api)
        """
    
        url = 'https://dapi.kakao.com/v2/local/search/address.json?query=' + address
        
        header = {'Authorization': 'KakaoAK ' + rest_api_token}
     
        r = requests.get(url, headers=header)
        
        if (r.status_code == 200) and len(r.json()["documents"])>0:
            lng = float(r.json()["documents"][0]["address"]['x'])
            lat = float(r.json()["documents"][0]["address"]['y'])
        else:
            return None
        return lat, lng
        
    return convert_address_to_coordinates

kakao_api_key = ""
convert_address_to_coordinates = set_rest_api(kakao_api_key)

In [100]:
find_coordinates(final_data.address.values[0])

('191450.441540966', '541949.351182625')

In [101]:
coordinates = [find_coordinates(i) for i in tqdm(final_data.address.values)]

  0%|          | 0/52 [00:00<?, ?it/s]

In [102]:
final_data0 = pd.concat([final_data, pd.DataFrame(coordinates, columns = ['x','y'])], axis=1)

In [103]:
def get_img_link(href_id,  dt='20241007'):
    time.sleep(1) 
    page_url = f"https://www.khug.or.kr/jeonse/web/s07/s070103.jsp?dt={dt}&{href_id}"
    html=requests.get(page_url)
    bs_test = BeautifulSoup(html.content, 'html.parser')
    img_src = bs_test.find(id = 'imgSor0')
    img_link = img_src.get('src')
    return img_link

In [104]:
final_data = final_data0.assign(img = lambda df: df.href_id.apply(lambda x: get_img_link(x)))

In [105]:
final_data.to_csv('data/data_241007.csv', index = False, encoding = 'utf-8-sig')

In [106]:
final_data.head(1)

,번호,공고일자,청약 접수기간,시도,시군구,주소,주택유형,전용면적(m2),임대보증금액,신청자수,href_id,address,x,y,img
0,52,20240830,2024.08.30. 10:00 ~ 2024.09.13. 17:00,서울특별시,서울 금천구,서울 금천구 독산동 962-32 ...,오피스텔(주거용),42.06,"207,000,000",1224,no=2021352488,서울 금천구 독산동 962-32,191450.441540966,541949.351182625,http://www.khug.or.kr/updata/khgc/khgccms/cms/...
